### Computational Complexity: Question 1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def operation_counts(m, features_list, epochs_batch=200, epochs_mb=400, epochs_sgd=1200):
    rows = []
    for k in features_list:
        ops_ne = m * (k**2) + (k**3)/3
        ops_svd = m * (k**2) + (k**3)
        ops_batch = 2 * epochs_batch * m * k
        ops_minibatch = 2 * epochs_mb * m * k
        ops_sgd = 2 * epochs_sgd * m * k
        rows.append({
            "Features (k)": k,
            "Closed-form NE (≈m·k²+⅓k³)": int(ops_ne),
            "Closed-form SVD (≈m·k²+k³)": int(ops_svd),
            "Batch GD (≈2E·m·k)": int(ops_batch),
            "Mini-batch GD (≈2E·m·k)": int(ops_minibatch),
            "SGD (≈2E·m·k)": int(ops_sgd)
        })

    return pd.DataFrame(rows)

In [ ]:
def main():
    m = 10_000
    features_list = [2, 5, 10, 15, 20]
    df = operation_counts(m, features_list)
    print("\nOperation Counts Comparison (m=10,000)\n")
    print(df.to_string(index=False))

In [ ]:
main()


Operation Counts Comparison (m=10,000)

 Features (k)  Closed-form NE (≈m·k²+⅓k³)  Closed-form SVD (≈m·k²+k³)  Batch GD (≈2E·m·k)  Mini-batch GD (≈2E·m·k)  SGD (≈2E·m·k)
            2                       40002                       40008             8000000                 16000000       48000000
            5                      250041                      250125            20000000                 40000000      120000000
           10                     1000333                     1001000            40000000                 80000000      240000000
           15                     2251125                     2253375            60000000                120000000      360000000
           20                     4002666                     4008000            80000000                160000000      480000000


### Computational Complexity: Question 2

In [ ]:
def gradient_descent_ops(m, k, lr=0.01, mode="batch", batch_size=None, tol=1e-3, max_epochs=10000):
    np.random.seed(0)
    X = np.random.randn(m, k)
    true_coef = np.random.randn(k)
    y = X @ true_coef + np.random.randn(m)*0.1
    theta = np.zeros(k)
    ops = 0
    epochs = 0
    while epochs < max_epochs:
        epochs += 1
        if mode == "batch":
            grad = (2/m) * X.T @ (X @ theta - y)
            new_theta = theta - lr * grad
            ops += 2*m*k
        elif mode == "minibatch":
            if batch_size is None:
                batch_size = 32
            for i in range(0, m, batch_size):
                X_batch = X[i:i+batch_size]
                y_batch = y[i:i+batch_size]
                grad = (2/len(X_batch)) * X_batch.T @ (X_batch @ theta - y_batch)
                new_theta = theta - lr * grad
                theta = new_theta
                ops += 2*len(X_batch)*k
            continue
        elif mode == "sgd":
            for i in range(m):
                xi = X[i].reshape(1, -1)
                yi = y[i]
                grad = 2 * xi.T @ (xi @ theta - yi)
                new_theta = theta - lr * grad.flatten()
                theta = new_theta
                ops += 2*k
            continue
        else:
            raise ValueError("Invalid mode")
        if np.linalg.norm(new_theta - theta) < tol:
            break
        theta = new_theta
    return epochs, ops

In [ ]:
def main1():
    m = 1000
    features_list = [2, 5, 10, 15, 20]

    results = []
    for k in features_list:
        batch_epochs, batch_ops = gradient_descent_ops(m, k, lr=0.01, mode="batch")
        mb_epochs, mb_ops = gradient_descent_ops(m, k, lr=0.01, mode="minibatch", batch_size=32)
        sgd_epochs, sgd_ops = gradient_descent_ops(m, k, lr=0.01, mode="sgd")

        results.append({
            "Features (k)": k,
            "Batch GD (epochs, ops)": (batch_epochs, batch_ops),
            "Mini-batch GD (epochs, ops)": (mb_epochs, mb_ops),
            "SGD (epochs, ops)": (sgd_epochs, sgd_ops)
        })

    df = pd.DataFrame(results)
    print(df.to_string(index=False))

In [ ]:
main1()

 Features (k) Batch GD (epochs, ops) Mini-batch GD (epochs, ops)  SGD (epochs, ops)
            2          (199, 796000)           (10000, 40000000)  (10000, 40000000)
            5         (200, 2000000)          (10000, 100000000) (10000, 100000000)
           10         (223, 4460000)          (10000, 200000000) (10000, 200000000)
           15         (230, 6900000)          (10000, 300000000) (10000, 300000000)
           20         (237, 9480000)          (10000, 400000000) (10000, 400000000)
